In [1]:
# Import packages
import csv                                                       # csv creation
import matplotlib.pyplot as plt                                  # Plotting
import numpy as np                                               # Numpy
import pandas as pd                                              # Pandas
from sklearn.linear_model import Ridge                           # Sklearn
from sklearn.model_selection import train_test_split
from scipy import stats                                          # Stats
import os                                                        # Context/resource management
import time
from contextlib import ExitStack
import gpytorch                                                  # Gpytorch/pytorch
import gpytorch.settings as gpts
import torch
from gpytorch.constraints import Interval
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import RBFKernel, RFFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch import Tensor
from botorch.fit import fit_gpytorch_mll                          # Botorch
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.models.transforms.outcome import Standardize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")

# 1. Generate synthetic data

In [2]:
# Function: gen_dat
# Description: Generate data following Section 5 of Cai, Wang, Zhang (2021)
# Inputs: n (int) -- sample size
#         p (int) -- dimension
# Outputs: X ((n,p) np.ndarray) -- design matrix of independent variable
#          y ((n,) np.ndarray) -- dependent variable
#          beta ((p,) np.ndarray) -- true coefficients
def gen_dat(n, p):
    # X_i ~iid Unif(-1/sqrt(p), 1/sqrt(p))
    X = np.random.uniform(low = -1/np.sqrt(p), high = 1/np.sqrt(p), size = (n, p))

    # epsilon_i ~iid N(0, 1)
    epsilon = np.random.normal(loc = 0, scale = 1, size = n)

    # beta ~ Unif(unit sphere)
    beta = np.random.normal(loc = 0, scale = 1, size = p)
    beta = beta / np.sqrt(np.dot(beta, beta))

    # y = X beta + epsilon
    y = X @ beta + epsilon
    
    return X, y, beta

# 2. Compute $\hat{\beta}^{\text{Ridge}}(\lambda)$
## Non-private: closed form solution
Note: Debugging statements (argument type checks) were obtained with the help of ChatGPT

In [3]:
# Function: ridge_closed
# Description: computes closed form solution of ridge regression problem: argmin ||Y - Xbeta||^2/n + ||beta||^2
# Inputs: X ((n,p) np.ndarray) -- design matrix of independent variable
#         y ((n,) np.ndarray) -- dependent variable
#         lambda_val ((1,) float) -- regularization parameter
# Outputs: beta_hat ((p,) np.ndarray) -- solution to ridge regression problem
def ridge_closed(X, y, lambda_val):
    # Ensure inputs are NumPy arrays
    X = np.asarray(X)
    y = np.asarray(y)

    # print(y.ndim); print(y.shape)
    # Check if lambda_val is a PyTorch tensor
    if isinstance(lambda_val, torch.Tensor):
        lambda_val = lambda_val.item()  # Convert to Python scalar
    
    # Ensure lambda_val is a scalar
    if not isinstance(lambda_val, (float, int)):
        raise TypeError("lambda_val must be a float, int, or a scalar PyTorch tensor.")
    
    # print(y.ndim); print(y.shape)
    # Check shapes
    if X.ndim != 2:
        raise ValueError("X must be a 2D array.")
    if y.ndim not in [1, 2]:
        raise ValueError("y must be a 1D or 2D array.")
    if y.ndim == 2 and y.shape[1] != 1:
        raise ValueError("y must have only one column if it is 2D.")

    # Ensure y is compatible with X
    if X.shape[0] != y.shape[0]:
        raise ValueError("Number of rows in X must match the number of elements in y.")
    
    # Compute the Ridge closed-form solution
    p = X.shape[1]
    n = X.shape[0]
    I = np.eye(p)
    beta_hat = np.linalg.inv((X.T @ X) / n + lambda_val * I) @ (X.T @ y) / n
    
    return beta_hat

## Private: Noisy gradient descent
Note: this is Algorithm 4.1 from Cai, Wang, Zhang (2021)

In [4]:
# ### Gradient descent for ridge regression

# ### Helper functions
# # Compute squared error loss
# def sq_loss(X, y, beta, lambda_val):
#     return np.dot((y - X.dot(beta)), (y - X.dot(beta))) + lambda_val * np.dot(beta, beta)

# # Compute gradient of ridge loss
# def grad(X, y, beta, lambda_val):
#      return (-1 / X.shape[0]) * (X.T @ (y - X.dot(beta))) + lambda_val * beta

# ### Gradient descent function
# def GD_ridge(X, y, grad, lambda_val, lr = 0.2, T = 500, init = None):
#     if init is None:
#         init = np.zeros(X.shape[1])
        
#     beta_t = init.astype(np.float64)
#     lambda_val = np.array(lambda_val)

#     losses = []
#     for t in range(T):
#         grad_update = grad(X = X, y = y, beta = beta_t, lambda_val = lambda_val) 
#         beta_t -= lr*grad_update
#         losses.append(sq_loss(X = X, y = y, beta = beta_t, lambda_val = lambda_val))
#         # print(sq_loss(X = X, y = y, beta = beta_t, lambda_val = lambda_val))
#         # print(grad(X = X, y = y, beta = beta_t, lambda_val = lambda_val))
#     return beta_t, grad_update, sq_loss(X = X, y = y, beta = beta_t, lambda_val = lambda_val), losses

In [5]:
# ### Gradient descent for ridge regression

# ### Helper functions
# # Compute squared error loss
# def sq_loss(X, y, beta, lambda_val):
#     return np.dot((y - X.dot(beta)), (y - X.dot(beta))) + lambda_val * np.dot(beta, beta)

# # Compute gradient of ridge loss
# def grad(X, y, beta, lambda_val):
#      return (-1 / X.shape[0]) * (X.T @ (y - X.dot(beta))) + lambda_val * beta

# ### Gradient descent function
# def GD_ridge(X, y, grad, lambda_val, lr = 0.2, T = 500, init = None):
#     if init is None:
#         init = np.zeros(X.shape[1])
        
#     beta_t = init.astype(np.float64)
#     lambda_val = np.array(lambda_val)

#     losses = []
#     for t in range(T):
#         grad_update = grad(X = X, y = y, beta = beta_t, lambda_val = lambda_val) 
#         beta_t -= lr*grad_update
#         losses.append(sq_loss(X = X, y = y, beta = beta_t, lambda_val = lambda_val))
        
#     return beta_t, grad_update, sq_loss(X = X, y = y, beta = beta_t, lambda_val = lambda_val), losses

In [6]:
# Compute gradient of ridge loss
def grad_update(X, y, beta, lambda_val, eta, epsilon, delta, B, R, C, T):
    # Generate weights with appropriate noise level
    priv_noise = np.square(eta)*2*np.square(B)*np.log(2*T/delta) / (np.square(n * epsilon / T))
    w = np.random.normal(loc = 0, scale = np.sqrt(priv_noise), size = p)

    y_clip = y
    inds = np.where(abs(y) > R)
    y_clip[inds] = np.sign(y_clip[inds])*R
    upd = beta - (eta / X.shape[0]) * (X.T @ (X.dot(beta) - y_clip)) + lambda_val*beta + w 
    
    upd_clip = upd
    inds = np.where(abs(upd) > C)
    upd_clip[inds] = np.sign(upd_clip[inds])*C
    
    return upd_clip
    
### In progress: private gradient descent for ridge regression
def GD_ridge(X, y, grad, lambda_val, eta, epsilon, delta, B, R, C, T, init):
    p = X.shape[1]
    if init is None:
        init = np.zeros(p)
        
    beta_t = init.astype(np.float64)

    if torch.is_tensor(lambda_val):
        lambda_val = lambda_val.cpu().numpy()
    
    # print(type(lambda_val))
    # lambda_val = np.array(lambda_val, dtype=np.float64)
    
    for t in range(T):    
        # Update beta by 
        beta_t = grad_update(X, y, beta_t, lambda_val, eta, epsilon, delta, B, R, C, T) 
    
    return beta_t

# 3. Compute validation loss (objective) of $\hat{\beta}^{\text{Ridge}}(\lambda)$

In [7]:
# Function: val_loss
# Description: Compute negative validation loss of beta estimated via ridge regression on training data
# Inputs: est_method (str) -- string ("private" or "non-private") indicating which method (private gradient descent 
#                             or non-private closed-form ridge regression
#         X_train ((n_train, p) np.ndarray) -- training design matrix
#         X_val ((n_val, p) np.ndarray) -- validation design matrix
#         y_train ((n_train,) np.ndarray) -- training dependent variable
#         y_val ((n_val,) np.ndarray) -- validation dependent variable
#         lambda_val ((1,) float) -- regularization parameter
# Outputs: neg_val_loss ((1,) float) -- negative quadratic loss on validation data
def val_loss(est_method, X_train, X_val, y_train, y_val, lambda_val, epsilon = None):
    # Set epsilon based on the method
    if est_method == "non-private":
        epsilon = 0.0
    elif epsilon is None:
        raise ValueError("You must provide a value for epsilon when method is not 'non-private'")
        
    # print("val loss" + str(epsilon))
    # print(est_method)
    # DP-gradient descent hyperparameters
    sigma = 1
    cx = 1.1                                           # Assumption (D1)
    L = 10                                             # Assumption (D2)
    c0 = 1.1                                           # Assumption (P1)
    # eta = p/(2*L)                                    # Theorem 4.2
    eta = 0.01
    R = sigma*np.sqrt(2*np.log(n))                     # Theorem 4.2
    T = int(np.ceil(8*(L**2)*np.log((c0**2)*n)))       # Theorem 4.2
    init=np.zeros(p)                                   # Theorem 4.2
    B = 4*(R + c0*cx)*cx                               # Lemma 4.2
    C = c0                                             # Lemma 4.2
    delta = 10/(n**1.1)
    
    # Estimate beta_hat on the training data according to the specified method
    if est_method == "private":
        beta_hat = GD_ridge(X_train, y_train, grad_update, lambda_val, eta, epsilon, delta, B, R, C, T, init)
    elif est_method == "non-private":
        # For a fixed lambda, estimate beta using ridge regression
        beta_hat = ridge_closed(X_train, y_train, lambda_val)
    else:
        # If a valid estimation method is not provided, throw an error
        raise ValueError("est_method must be either 'private' or 'non-private'")
    
    # Compute negative squared error loss on validation data
    neg_val_loss = (-1 / X_val.shape[0]) * np.dot((y_val - X_val.dot(beta_hat)), (y_val - X_val.dot(beta_hat)))
    
    return neg_val_loss

# 4. Select $\lambda$ by optimizing validation loss
## Non-private: grid search

In [8]:
# Function: min_val_loss
# Description: minimizes validation loss with respect to lambda via grid search
# Inputs: X_train ((n_train, p) np.ndarray) -- training design matrix
#         X_val ((n_val, p) np.ndarray) -- validation design matrix
#         y_train ((n_train,) np.ndarray) -- training dependent variable
#         y_val ((n_val,) np.ndarray) -- validation dependent variable
#         lambda_vals ((grid_size,) np.ndarray) -- grid values
# Outputs: lambda_vals ((grid_size,) np.ndarray) -- grid values (for plotting purposes)
#          losses ((grid_size,) np.ndarray) -- values of negative validation loss evaluated over grid values (for plotting purposes)
#          lambda_hat ((1,) float) -- lambda that maximizes negative loss 
def min_val_loss(est_method, X_train, X_val, y_train, y_val, lambda_vals, epsilon = None):
    # Set epsilon based on the method
    if est_method == "non-private":
        epsilon = 0.0
    elif epsilon is None:
        raise ValueError("You must provide a value for epsilon when method is not 'non-private'")
        
    # print("NP" + str(epsilon))
    # print(est_method)
    # Compute the validation loss over the specified grid of lambda values
    losses = []
    for l in range(lambda_vals.shape[0]):
        losses.append(val_loss(est_method, X_train, X_val, y_train, y_val, lambda_vals[l], epsilon))

    # Select the value of lambda that maximizes the negative validation loss
    max_value = max(losses)
    lambda_hat = lambda_vals[losses.index(max_value)]
    
    return lambda_vals, losses, lambda_hat

## Private 1: random grid search
Notes:
1. The implementation of the truncated negative binomial distribution is adapted from user hilberts_drinking_problem's answer on this thread: https://stackoverflow.com/questions/67601496/generating-truncated-negative-binomial-distribution-in-python#comment119488769_67601496
2. The implementation of private grid search is from Theorem 2 of Papernot and Steinke (2021)

In [9]:
# Function: trunc_Nbinom
# Description: Generates replicates from the truncated negative biomial distribution (i.e., the negative bimomial distribution
#              supported on {1,2,...} as opposed to {0,1,2,...})
# Inputs: ntrial (int) number of trials parameter of negative binomial distribution
#         sprob (int) success probability parameter of negative binomial distribution
#         size (int) -- number of samples
# Outputs: samples ((size,) np.ndarray) -- replicates from the truncated Nbinom(n,p)
def trunc_Nbinom(ntrial, sprob, size):
    # Define the support of the distribution to be {1,2,...}
    support = np.delete(np.arange(ntrial), 0)

    # Evaluate the Nbinom(n,p) pmf over the support {1,2,...} and normalize the values to define a valid pmf
    probs = stats.nbinom.pmf(support, ntrial, sprob)
    probs /= probs.sum()

    # Generate samples according to the pmf defined above
    samples = np.random.choice(support, size=size, p=probs)
    
    return samples

# Function: PS_min_val_loss
# Description: implementatiom of private grid search from Theorem 2 of Papernot and Steinke (2021)
# Inputs: X_train ((n_train, p) np.ndarray) -- training design matrix
#         X_val ((n_val, p) np.ndarray) -- validation design matrix
#         y_train ((n_train,) np.ndarray) -- training dependent variable
#         y_val ((n_val,) np.ndarray) -- validation dependent variable
#         sprob (int) -- success probability parameter of negative binomial random variable
#         lambda_vals ((grid_size,) np.ndarray) -- grid values
def PS_min_val_loss(est_method, X_train, X_val, y_train, y_val, sprob, lambda_vals, epsilon=None):
    # Set epsilon based on the method
    if est_method == "non-private":
        epsilon = 0.0
    elif epsilon is None:
        raise ValueError("You must provide a value for epsilon when method is not 'non-private'")
        
    # print("PS" + str(epsilon))
    # print(est_method)
    # Select the total number of times to evaluate the loss
    K = trunc_Nbinom(lambda_vals.shape[0], sprob, 1).item()

    losses = []
    lambda_vals_sel = []
    for l in range(K):
        # select a point from lambda_vals uniformly at random
        rand_ind = np.random.randint(lambda_vals.shape[0], size = 1)
        lambda_sel = lambda_vals[rand_ind].item()
        lambda_vals_sel.append(lambda_sel)

        # evaluate the validation loss at that point
        losses.append(val_loss(est_method, X_train, X_val, y_train, y_val, lambda_sel, epsilon))
    
    # Select the value of lambda that maximizes the negative validation loss
    max_value = max(losses)
    lambda_hat = lambda_vals_sel[losses.index(max_value)]
    
    return lambda_vals_sel, losses, lambda_hat

## Private 2: Bayesian optimization with Thompson sampling acquisition
Notes:
1. This implementation is adapted from the following Botorch tutorial: https://botorch.org/tutorials/thompson_sampling
2. Debugging statements (argument type checks) were obtained with the help of ChatGPT

In [10]:
# Function: generate_batch
# Description: Thompson sampling acquisition
# Inputs: X (torch.tensor) -- evaluation points of objective function (so far)
#         Y (torch.tensor) -- objective evaluations (so far)
#         batch_size -- number of posterior samples to obtain
#         n_candidates (int) -- number of places to evaluate surrogate
#         X_train ((n_train, p) np.ndarray) -- training design matrix
#         X_val ((n_val, p) np.ndarray) -- validation design matrix
#         y_train ((n_train,) np.ndarray) -- training dependent variable
#         y_val ((n_val,) np.ndarray) -- validation dependent variable
#         lambda_vals ((grid_size,) np.ndarray) -- grid values
# Outputs: X_next ((batch_size,) torch.tensor) -- next point to evaluate objective based on Thompson sampling
def generate_batch(
    X: Tensor,
    Y: Tensor,
    batch_size: int,
    n_candidates: int,
    X_train: np.ndarray, 
    X_val: np.ndarray, 
    y_train: np.ndarray, 
    y_val: np.ndarray,
    lambda_vals: torch.tensor
) -> Tensor:
    # Ensure that candidate values are within the bounds of the lambda_vals
    assert X.min() >= lambda_vals.min() and X.max() <= lambda_vals.max() and torch.all(torch.isfinite(Y))

    # Ensure X is a PyTorch tensor and reshape if necessary
    if isinstance(X, np.ndarray):
        X = torch.tensor(X, dtype=torch.float64)
    if X.ndim == 1:
        X = X.unsqueeze(-1)
    
    # Ensure Y is a PyTorch tensor and reshape if necessary
    if isinstance(Y, np.ndarray):
        Y = torch.tensor(Y, dtype=torch.float64)
    if Y.ndim == 1:
        Y = Y.unsqueeze(-1)
    
    # Check for dimensional consistency
    if X.shape[0] != Y.shape[0]:
        raise ValueError(f"Mismatch in sample sizes: X has {X.shape[0]}, Y has {Y.shape[0]}")

    # Specify and fit GP surrogate model
    base_kernel = RBFKernel(ard_num_dims=X.shape[-1])
    covar_module = ScaleKernel(base_kernel)
    model = SingleTaskGP(train_X=X, train_Y=Y, covar_module=covar_module, outcome_transform=Standardize(m=1))
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    X_cand = lambda_vals
    # Thompson sample
    with ExitStack() as es:
        es.enter_context(gpts.max_cholesky_size(float("inf")))
        es.enter_context(torch.no_grad())
        
        thompson_sampling = MaxPosteriorSampling(model=model, replacement=False)
        X_next = thompson_sampling(X_cand, num_samples=batch_size)

    return X_next

# Function: run_optimization
# Description: Bayesian optimization loop with Thompson sampling acquisition
# Inputs: n_candidates (int) -- number of places to evaluate surrogate
#         n_init (int) -- Number of initial points of objective to evaluate
#         max_evals (int) -- Number of BO iterations
#         batch_size (int) -- Number of samples to obtain via Thompson sampling
#         lambda_vals ((grid_size, 1) torch.tensor) -- grid values
def run_optimization(
    est_method: str,
    n_candidates: int,
    n_init: int,
    max_evals: int,
    batch_size: int,
    X_train: np.ndarray, 
    X_val: np.ndarray, 
    y_train: np.ndarray, 
    y_val: np.ndarray,
    lambda_vals: torch.tensor,
    epsilon: float = None,
) -> tuple[Tensor, Tensor]:
    # Set epsilon based on the method
    if est_method == "non-private":
        epsilon = 0.0
    elif epsilon is None:
        raise ValueError("You must provide a value for epsilon when method is not 'non-private'")
        
    # print(est_method)
    if lambda_vals.ndim == 1:
        lambda_vals = lambda_vals.unsqueeze(-1)
    
    # First optimization step: sample a smaller subset of lambda_vals points uniformly (initial points)
    inds_init = torch.from_numpy(np.random.randint(lambda_vals.size()[0], size = n_init))
    X = lambda_vals[inds_init]
    # Evaluate at initial points
    # print("BO" + str(epsilon))
    # print("BOtest1")
    Y = torch.tensor(
        [val_loss(est_method, X_train, X_val, y_train, y_val, x, epsilon) for x in X], dtype=dtype, device=device
    ).unsqueeze(-1)
    # print("BOtest2")
    # Print best value of those evaluated on initial points
    # print(f"{len(X)}) Best value: {Y.max().item():.2e}")

    # BO loop
    while len(X) < max_evals:
        # Create a batch
        start = time.monotonic()
        X_next = generate_batch(
            X=X,
            Y=Y,
            batch_size=min(batch_size, max_evals - len(X)),
            n_candidates=n_candidates,
            lambda_vals=lambda_vals,
            X_train=X_train,
            X_val=X_val,
            y_train=y_train,
            y_val=y_val
        )
        end = time.monotonic()
        # print(f"Generated batch in {end - start:.1f} seconds")
        
        Y_next = torch.tensor(
            [val_loss(est_method, X_train, X_val, y_train, y_val, x, epsilon) for x in X_next], dtype=dtype, device=device
        ).unsqueeze(-1)
        # Append data
        X = torch.cat((X, X_next), dim=0)
        Y = torch.cat((Y, Y_next), dim=0)
        
        # print(f"{len(X)}) Best value: {Y.max().item():.2e}")
    
    lambda_hat = X[Y.argmax().item()].item()
    
    return X, Y, lambda_hat

# 5. Compute estimation error

# 6. Simulation loop
Note: simulation loop includes code to help create and save .csv files that was created with the help of ChatGPT

In [11]:
def sim_loop(est_method, n, p, lambda_vals, sprob, epsilon=None):
    # Set epsilon based on the method
    if est_method == "non-private":
        epsilon = 0.0
    elif epsilon is None:
        raise ValueError("You must provide a value for epsilon when method is not 'non-private'")
        
    # print("sim loop" + str(epsilon))
    # print(est_method)
    sigma = 1
    cx = 1.1                                           # Assumption (D1)
    L = 10                                             # Assumption (D2)
    c0 = 1.1                                           # Assumption (P1)
    # eta = p/(2*L)                                    # Theorem 4.2
    eta = 0.01
    R = sigma*np.sqrt(2*np.log(n))                     # Theorem 4.2
    T = int(np.ceil(8*(L**2)*np.log((c0**2)*n)))       # Theorem 4.2
    init=np.zeros(p)                                   # Theorem 4.2
    B = 4*(R + c0*cx)*cx                               # Lemma 4.2
    C = c0                                             # Lemma 4.2
    delta = 10/(n**1.1)

    # Generate data
    X, y, beta = gen_dat(n, p)
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
    # print(X_train.shape)
    # Method 1: non-private (minimize validation loss)
    NP_lambda_vals, NP_val, NP_lambda_hat = min_val_loss(est_method, X_train, X_val, y_train, y_val, lambda_vals, epsilon)
    # print("X_train1")
    if est_method == "private":
        NP_beta_hat = GD_ridge(X_train, y_train, grad_update, NP_lambda_hat, eta, epsilon, delta, B, R, C, T, init)
    else: 
        NP_beta_hat = ridge_closed(X_train, y_train, NP_lambda_hat)    
    
    NP_est_err = np.dot(NP_beta_hat - beta, NP_beta_hat - beta)
    
    # Method 2: private method by Papernot and Steinke (2021)
    PS_lambda_vals, PS_val, PS_lambda_hat = PS_min_val_loss(est_method, X_train, X_val, y_train, y_val, sprob, lambda_vals, epsilon)    

    if est_method == "private":
        PS_beta_hat = GD_ridge(X_train, y_train, grad_update, PS_lambda_hat, eta, epsilon, delta, B, R, C, T, init)
    else:
        PS_beta_hat = ridge_closed(X_train, y_train, PS_lambda_hat)
    
    PS_est_err = np.dot(PS_beta_hat - beta, PS_beta_hat - beta)
    # print("X_train2")
    # Method 3: BO with TS acquisition
    # BO with TS arguments
    batch_size = 1
    n_init = 10
    # max_evals = 50
    max_evals = trunc_Nbinom(lambda_vals.shape[0], sprob, 1)
    N_CAND = 10_00 if not SMOKE_TEST else 10
    grid = torch.from_numpy(lambda_vals)
    shared_args = {
            "est_method": est_method,
            "n_candidates": N_CAND,
            "n_init": n_init,
            "max_evals": max_evals,
            "batch_size": batch_size,
            "X_train": X_train, 
            "X_val": X_val, 
            "y_train": y_train, 
            "y_val": y_val,
            "lambda_vals": grid,
            "epsilon": epsilon
        }
    # print("X_train2.5")
    BO_lambda_vals, BO_vals, BO_lambda_hat = run_optimization(**shared_args)
    # print("X_train3")
    if est_method == "private":
        BO_beta_hat = GD_ridge(X_train, y_train, grad_update, BO_lambda_hat, eta, epsilon, delta, B, R, C, T, init)
    else:
        BO_beta_hat = ridge_closed(X_train, y_train, BO_lambda_hat)
    
    BO_est_err = np.dot(BO_beta_hat - beta, BO_beta_hat - beta)
    # print("X_train4")
    out = [NP_lambda_hat, PS_lambda_hat, BO_lambda_hat, 
           np.min(NP_val), np.min(PS_val), np.min(BO_vals.numpy()), 
           NP_est_err, PS_est_err, BO_est_err]

    return out

In [12]:
# Simulation loop function
def simulation_loop(est_method, csv_path, total_iterations, n, p, lambda_vals, sprob, epsilon=None):
    # print("est method:" + est_method)
    # print("path:" + csv_path)
    # print("totit" + str(total_iterations))
    # print("n=" + str(n))
    # print("p=" + str(p))
    # print(lambda_vals)
    # print("sprob" + str(sprob))
    # print(epsilon)
    
    # Set epsilon based on the method
    if est_method == "non-private":
        epsilon = 0.0
    elif epsilon is None:
        raise ValueError("You must provide a value for epsilon when method is not 'non-private'")
        
    try:
        # Check if the file exists, and create it if it does not
        if not os.path.exists(csv_path):
            print(f"File '{csv_path}' does not exist. Creating a new file with a default header.")
            with open(csv_path, mode="w", newline="") as csv_file:
                csv_writer = csv.writer(csv_file)
                # Write a default header
                csv_writer.writerow(["NP_lambda_hat", "PS_lambda_hat", "BO_lambda_hat", 
                                     "NP_min_val", "PS_min_val", "BO_min_val", 
                                     "NP_est_err", "PS_est_err", "BO_est_err"])

        # Append new rows for the specified number of iterations
        print("Appending new simulation data...")
        with open(csv_path, mode="a", newline="") as csv_file:
            csv_writer = csv.writer(csv_file)
            for iteration in range(1, total_iterations + 1):
                new_row = sim_loop(est_method, n, p, lambda_vals, sprob, epsilon)
                csv_writer.writerow(new_row)
                # print(f"Added row for iteration {iteration}: {new_row}")
                if (iteration%10) == 0:
                    print(f"     Added row for iteration {iteration}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [13]:
# # Common parameters
# nsim = 50
# p = 10
# sprob = 0.6

# # Simulation-specific parameters
# params = [["non-private", 200, 10], ["private", 200, 100, 5]]

# # Run simulations for specified string of parameters
# for i in range(len(params)):
#     params_sim = params[i]
#     est_method = params_sim[0]
#     n = params_sim[1]
#     ngrid = params_sim[2]

#     if est_method == "private":
#         epsilon = params_sim[3]
#         print("Starting simulation with: n=" + str(n) + ", grid=" + str(ngrid) + ", epsilon=" + str(epsilon))
#         file = "exp1_" + str(n) + "_" + str(epsilon*100) + "_" + str(ngrid) +".csv"
#         simulation_loop(est_method, file, nsim, n, p, np.linspace(0,1,ngrid), sprob, epsilon)
#     else:
#         print("Starting simulation with: n=" + str(n) + ", grid=" + str(ngrid) + est_method)
#         file = "exp1_" + str(n) + "_" + est_method + "_" + str(ngrid) +".csv"
#         simulation_loop(est_method, file, nsim, n, p, np.linspace(0,1,ngrid), sprob)

Starting simulation with: n=200, grid=10non-private
est method:non-private
path:exp1_200_non-private_10.csv
totit50
n=200
p=10
[0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
sprob0.6
None
Appending new simulation data...
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2
X

/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  lambda_val = np.array(lambda_val)


BOtest2


/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  lambda_val = np.array(lambda_val)
/Users/ruchiraray/Documents/Project/venv/lib/python3.11/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-k

X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2


...

X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2


/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  lambda_val = np.array(lambda_val)
/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  lambda_val = np.array(lambda_val)
/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy ke

X_train3
X_train4
(140, 10)
X_train1
X_train2
X_train2.5
BOtest1
BOtest2


/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  lambda_val = np.array(lambda_val)
/var/folders/2j/ctc2_zx906ng15d6rg25fpt80000gn/T/ipykernel_47935/3387318527.py:26: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  lambda_val = np.array(lambda_val)
/Users/ruchiraray/Documents/Project/venv/lib/python3.11/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter o

KeyboardInterrupt: 

In [17]:
# simulation_loop(est_method, file, nsim, n, p, np.linspace(0,1,ngrid), sprob, epsilon)

Appending new simulation data...
An error occurred: name 'X_train' is not defined


In [13]:
# n = 1000
# p = 10
# X, y, beta = gen_dat(n, p)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
# # lambda_val = 0.01
# # est_method = "private"
# sprob = 0.6
# lambda_vals = np.linspace(0,1,100)

# total_iterations = 2

# # sim_loop("non-private", n, p, lambda_vals, sprob)
# # sim_loop("private", n, p, lambda_vals, sprob, epsilon=0.5)

# simulation_loop("non-private", "test1.csv", total_iterations, n, p, lambda_vals, sprob)
# simulation_loop("private", "test2.csv", total_iterations, n, p, lambda_vals, sprob, epsilon=0.5)

# # max_evals = 50
# N_CAND = 10_00 if not SMOKE_TEST else 10
# max_evals = trunc_Nbinom(lambda_vals.shape[0], sprob, 1)
# lambda_vals_torch = torch.from_numpy(lambda_vals)

# shared_args = {
#         "est_method": "private",
#         "n_candidates": N_CAND,
#         "n_init": 10,
#         "max_evals": max_evals,
#         "batch_size": 1,
#         "lambda_vals": lambda_vals_torch,
#         "epsilon": 0.5
#     }
# BO_lambda_vals, BO_vals, BO_lambda_hat = run_optimization(**shared_args)
# plt.scatter(BO_lambda_vals, BO_vals)

# shared_args = {
#         "est_method": "non-private",
#         "n_candidates": N_CAND,
#         "n_init": 10,
#         "max_evals": max_evals,
#         "batch_size": 1,
#         "lambda_vals": lambda_vals_torch,
#     }
# BO_lambda_vals, BO_vals, BO_lambda_hat = run_optimization(**shared_args)
# plt.scatter(BO_lambda_vals, BO_vals)

# lambda_vals_sel, losses, lambda_hat = PS_min_val_loss("private", X_train, X_val, y_train, y_val, sprob, lambda_vals, epsilon=0.5)
# plt.scatter(lambda_vals_sel, losses)
# lambda_vals_sel, losses, lambda_hat = PS_min_val_loss("non-private", X_train, X_val, y_train, y_val, sprob, lambda_vals)
# plt.scatter(lambda_vals_sel, losses)

# lambda_vals, losses, lambda_hat = min_val_loss("private", X_train, X_val, y_train, y_val, lambda_vals, epsilon=0.5)
# plt.scatter(lambda_vals, losses)
# lambda_vals, losses, lambda_hat = min_val_loss("non-private", X_train, X_val, y_train, y_val, lambda_vals)
# plt.scatter(lambda_vals, losses)

# lambda_val = 1.0
# print(val_loss("private", X_train, X_val, y_train, y_val, lambda_val, epsilon=3))
# print(val_loss("non-private", X_train, X_val, y_train, y_val, lambda_val))

In [14]:
# X, y, beta = gen_dat(10000, 3)
# # print(X.shape); print(y.shape); print(beta.shape)
# train_test_split(X, y, test_size=0.3)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
# ridge_closed(X_train, y_train, 1.0)

# batch_size = 1
# n_init = 10
# # max_evals = 50
# N_CAND = 10_00 if not SMOKE_TEST else 10
# lambda_vals = np.linspace(0,1,100)
# max_evals = trunc_Nbinom(lambda_vals.shape[0], sprob, 1)
# grid = torch.from_numpy(lambda_vals)
# est_meth = "private"
# shared_args = {
#         "est_method": est_meth,
#         "n_candidates": N_CAND,
#         "n_init": n_init,
#         "max_evals": max_evals,
#         "batch_size": batch_size,
#         "lambda_vals": grid
#     }
# BO_lambda_vals, BO_vals, BO_lambda_hat = run_optimization(**shared_args)

# plt.scatter(BO_lambda_vals, BO_vals)

In [15]:
# n = 100000
# p = 10
# X, y, beta = gen_dat(n, p)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
# lambda_vals = np.linspace(0,1,100)

# sigma = 1
# cx = 1.1                                           # Assumption (D1)
# L = 10                                             # Assumption (D2)
# c0 = 1.1                                           # Assumption (P1)
# # eta = p/(2*L)                                    # Theorem 4.2
# eta = 0.01
# R = sigma*np.sqrt(2*np.log(n))                     # Theorem 4.2
# T = int(np.ceil(8*(L**2)*np.log((c0**2)*n)))       # Theorem 4.2
# init=np.zeros(p)                                   # Theorem 4.2
# B = 4*(R + c0*cx)*cx                               # Lemma 4.2
# C = c0                                             # Lemma 4.2
# delta = 10/(n**1.1)
# epsilon=2

# # lambda_vals, losses, lambda_hat = min_val_loss(X_train, X_val, y_train, y_val, lambda_vals)
# # plt.scatter(lambda_vals, losses)
# sprob=0.6
# batch_size = 1
# n_init = 10
# # max_evals = 50
# N_CAND = 10_00 if not SMOKE_TEST else 10
# max_evals = trunc_Nbinom(lambda_vals.shape[0], sprob, 1)
# grid = torch.from_numpy(lambda_vals)
# est_meth = "non-private"
# shared_args = {
#         "est_method": est_meth,
#         "n_candidates": N_CAND,
#         "n_init": n_init,
#         "max_evals": max_evals,
#         "batch_size": batch_size,
#         "lambda_vals": grid
#     }
# BO_lambda_vals, BO_vals, BO_lambda_hat = run_optimization(**shared_args)
# plt.scatter(BO_lambda_vals, BO_vals)